<a href="https://colab.research.google.com/github/sipocz/DataMiner/blob/main/DataMiner_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from bs4 import  BeautifulSoup

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
path="/content/drive/MyDrive/001_METU/2. félév/dolgozat/adatok/"

In [ ]:
fname="Eseményfeladatok _ RG - IT Service Management.html"


In [ ]:
htmltext=open(path+fname)

In [ ]:
ht=htmltext.read()

In [ ]:
print("------------------------------------------------")

soup=BeautifulSoup(ht,"html.parser")
table_rows1=soup.find_all("tr", class_="list_row list_odd is-selected")
table_rows2=soup.find_all("tr", class_="list_row list_even is-selected")
table_rows=table_rows1+table_rows2

In [ ]:
len(table_rows)

In [ ]:
a_row=table_rows[2].find_all("td",class_="vt")

In [ ]:
a_row[7].div["timeago"]

In [ ]:
a_row[7].div.text

In [ ]:
def print_a_row(a_row,to_print=False):
    task_id=a_row[0].a["aria-label"].split(": ")[1]
    task_link=a_row[0].a["href"]
    task_description=a_row[1].text
    inc_name=a_row[6].a["aria-label"].split(": ")[1]
    inc_link=a_row[6].a["href"]
    t_creation=a_row[8].div.text
    t_start=a_row[7].div.text
    t_end=a_row[9].div.text
    try:
        full_text=a_row[10]["title"]
    except:
        full_text="None"

    if to_print:
        print("taskid:",task_id,"\ntask_link: ",task_link)
        print(a_row[1].text)

        print("incident name:",inc_name)
        print("incident link:",inc_link )
    
        print("Creation time:",t_creation)
        print("Start time:",t_start)
        print("End time",t_end)
    


         
        print("Text descriptiopn:",full_text)
        
        print("__________________________")
    return([task_id,task_link,task_description,inc_name,inc_link,t_creation,t_start,t_end,full_text])

In [ ]:
data_list=[]
for a_row_e in table_rows:
    a_row=a_row_e.find_all("td",class_="vt")
    data_list.append(print_a_row(a_row,to_print=False))


In [ ]:
print (len(data_list))

In [ ]:
fname="Incidents _ RG - IT Service Management.html"

In [ ]:
htmltext=open(path+fname)

In [ ]:
ht=htmltext.read()
soup=BeautifulSoup(ht,"html.parser")


In [ ]:
table_rows1=soup.find_all("tr",  class_="list_row list_odd ")
table_rows2=soup.find_all("tr",  class_="list_row list_even ")
table_rows=table_rows1+table_rows2

In [ ]:
len(table_rows)

In [ ]:
a_row=table_rows[2].find_all("td",class_="vt")

In [ ]:
def print_incident_row(a_row,to_print=False):
    task_id=a_row[0].a["aria-label"].split(": ")[1]
    task_link="https://richterprod.service-now.com/"+a_row[0].a["href"]
    task_description=a_row[4].text
    inc_name=task_id
    inc_link=task_link
    t_creation=a_row[11].div.text
    t_start=a_row[10].div.text
    t_end=a_row[12].div.text
    try:
        full_text1=a_row[4].text
    except:
        full_text1="None"

    
    try:
        full_text2=a_row[14]["title"]
    except:
        full_text2="None"
    full_text=full_text1+"\n"+full_text2
    if to_print:
        print("taskid:",task_id,"\ntask_link: ",task_link)
        print("task description:",task_description)

        print("incident name:",inc_name)
        print("incident link:",inc_link )
    
        print("Creation time:",t_creation)
        print("Start time:",t_start)
        print("End time",t_end)
    


         
        print("Text description:",full_text)
        
        print("__________________________")
    return([task_id,task_link,task_description,inc_name,inc_link,t_creation,t_start,t_end,full_text])
    

In [ ]:
data_list2=[]
for a_row_e in table_rows:
    a_row=a_row_e.find_all("td",class_="vt")
    data_list2.append(print_incident_row(a_row,to_print=False))

In [ ]:
len(data_list2)


In [ ]:
all_data_list=data_list+data_list2


In [ ]:
len(all_data_list)

In [ ]:
import pandas as pd


In [ ]:
df=pd.DataFrame(all_data_list,columns=["Task","TaskLink","Desc","Inc","IncLink","Creation","Start","End","LongDesc"])

In [ ]:
df[['Start', 'End', 'Creation']] = df[['Start', 'End', 'Creation']].apply(pd.to_datetime)

In [ ]:
df["End_Delta"]=df["End"]-df["Start"]
df["End_Delta_sec"]=pd.to_numeric(df["End_Delta"]/1000000000)

In [ ]:
df["Start_Delta"]=df["Start"]-df["Creation"]
df["Start_Delta_sec"]=pd.to_numeric(df["Start_Delta"]/1000000000)

In [ ]:
df.sort_values(["Creation"],inplace=True,ignore_index=True)

In [ ]:
df.reindex()

In [ ]:
df.plot(y="Start_Delta_sec")

In [ ]:
for i in range(len(df)):
    print(i,df.loc[i].TaskLink)

In [ ]:
fname="INC0188514 _ Incidens _ RG - IT Service Management.html"
path="/content/drive/MyDrive/001_METU/2. félév/dolgozat/adatok/inc/"


In [ ]:
htmltext=open(path+fname)
ht=htmltext.read()
soup=BeautifulSoup(ht,"html.parser")

In [ ]:
print("------------------------------------------------")

soup=BeautifulSoup(ht,"html.parser")
table_rows=soup.find_all("li", class_="h-card h-card_md h-card_comments")


In [ ]:
table_rows[7]

In [ ]:
def find_elements(el):
    temp=el.findChildren()
    return len(temp)

In [ ]:
def incident_record_elements(incident_record, to_print=False):
    out=[]
    for child in incident_record.descendants:
        try:
            if find_elements(child)==0:
                if to_print:
                    print(child.text)
                out.append(child.text)
        except:
            pass
    return(out)
            

In [ ]:
def get_all_html(path:str):
    import os
    files=os.scandir(path=path,)
    o=[a.name for a in files if "html" in a.name ]
    o.sort()
    return o


In [ ]:
html_files = get_all_html(path)

In [ ]:
html_files

In [ ]:
def critical_stepps(incident_record_list):
    '''
    Adatok strukturálása
    '''
    o=[]
    for incident_record in incident_record_list:
        incident_record_list=incident_record_elements(incident_record)
        
        if "ServiceNow - Incidens került a csoportodra | Incident has been assigned to your group" in incident_record_list:
            o1=f"0:csoporton:{incident_record_list[3]}"
            print(o1)
            o.append(o1)
        
        if "ServiceNow - Incidens került a nevedre | Incident has been assigned to you" in incident_record_list:
            o2=f"1:kollégán:{incident_record_list[3]}"
            print(o2)
            o.append(o2)
        if "Megoldott (véglegesen)" in incident_record_list:
            o3=f"2:megoldott:{incident_record_list[5]}"
            print(o3)
            o.append(o3)
        
        if "Megoldott (kerülő megoldás)" in incident_record_list:
            o4=f"2:megoldott (kerülővel):{incident_record_list[5]}"
            print(o4)
            o.append(o4)
        
        if "Nem megoldott (nem reprodukálható)" in incident_record_list:
            o5=f"2:megoldott (Nem megoldott de nem reprodukálható)):{incident_record_list[5]}"
            print(o5)
            o.append(o5)
    return o   
        
            


In [ ]:
critical_stepps(table_rows)

In [ ]:
def list_all_incident(path:str):
    html_files=get_all_html(path)
    #print(type(html_files))
    i=0
    out=[]
    for fname in html_files:
        out_rec=[]
        i+=1
        inc_name=fname.split(" ")[0]
        out_rec.append(inc_name)
        print(i,inc_name)
        htmltext=open(path+fname)
        ht=htmltext.read()
        soup=BeautifulSoup(ht,"html.parser")
        
        
        table_rows=soup.find_all("li", class_="h-card h-card_md h-card_comments")

        ret=critical_stepps(table_rows)
        out_rec.append(len(ret))
        out_rec.append(ret)
        print("------------------------------------------------")
        out.append(out_rec)
    return out

In [ ]:
o=list_all_incident(path)

In [ ]:
o

In [ ]:
incident_record_list=incident_record_elements(table_rows[4])

In [ ]:
incident_record_list

In [ ]:
'Megoldott (véglegesen) ' in incident_record_list

In [ ]:
find_elements(table_rows[0])

In [ ]:
path="/content/drive/MyDrive/001_METU/2. félév/dolgozat/adatok/task/"

In [ ]:
html_files = get_all_html(path)

In [ ]:
list_all_incident(path)

1 TASK0216008
------------------------------------------------
2 TASK0225687
------------------------------------------------
3 TASK0225759
------------------------------------------------
4 TASK0227851
------------------------------------------------
5 TASK0230843
------------------------------------------------
6 TASK0232153
------------------------------------------------
7 TASK0233522
------------------------------------------------
8 TASK0234168
------------------------------------------------
9 TASK0234169
------------------------------------------------
10 TASK0234244
------------------------------------------------
11 TASK0235078
------------------------------------------------
12 TASK0241059
------------------------------------------------
13 TASK0241109
------------------------------------------------
14 TASK0242057
------------------------------------------------
15 TASK0243081
------------------------------------------------
16 TASK0244980
----------------------------------

In [ ]:
path="/content/drive/MyDrive/001_METU/2. félév/dolgozat/adatok/emails/"

In [ ]:
def fname_converter(fname:str, to_print=False):
    o=fname.split(".")[0].split("_")
    date_s=o[0].split("-")
    time_s=o[1].split("-")
    o=date_s+time_s
    out=f"{o[0]}-{o[1]}-{o[2]} {o[3]}:{o[4]}:{o[5]}"    
    if to_print:
        print(out)
    return(out)

In [ ]:
fname_converter("2022-11-10_05-52-13.txt")

In [ ]:
import os
import time
email_files=os.scandir(path=path,)
email_files_data=[fname_converter(a.name) for a in email_files ]

In [ ]:
email_files_data.sort()

In [ ]:
email_files_data